In [10]:
import dask
import dask.dataframe as dd

dask.config.set({"dataframe.convert-string": False})

dataset_path = "./smadex-challenge-predict-the-revenue/train/train"
filters = [("datetime", ">=", "2025-10-01-00-00"), ("datetime", "<", "2025-10-01-01-00")]

ddf = dd.read_parquet(
    dataset_path,
    filters=filters
)


In [11]:
len(ddf.columns)

85

In [12]:
nrows = ddf.shape[0].compute()
nrows

121887

In [13]:
ddf.head()

,buyer_d1,buyer_d7,buyer_d14,buyer_d28,buy_d7,buy_d14,buy_d28,iap_revenue_d7,iap_revenue_d14,iap_revenue_d28,...,user_bundles_l28d,weekend_ratio,weeks_since_first_seen,wifi_ratio,whale_users_bundle_num_buys_prank,whale_users_bundle_revenue_prank,whale_users_bundle_total_num_buys,whale_users_bundle_total_revenue,row_id,datetime
0,0,1,1,1,1,1,1,2.147718,2.147718,2.147718,...,"[88981729bd5c1e5aea9ada4bce00a2531e9e98f7, 25c...",0.019802,6.0,0.913366,None,None,None,None,819ecc0e-1a97-43ed-83f6-b9ede4f7fc48,2025-10-01-00-00
1,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,None,NaN,NaN,NaN,None,None,None,None,0a7fbf18-5041-42af-bd0a-0cb6586b8598,2025-10-01-00-00
2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[6506b7e0a24666debd08f74266800f2eb154df5a, 150...",0.399021,6.0,0.999388,None,None,None,None,fc1a2689-b136-4ffa-b23b-9d8215bd720f,2025-10-01-00-00
3,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[2b472e3dc96f1847490d7411b25e12ed417b9714, 3ba...",0.121547,6.0,1.000000,None,None,None,None,0340fcc6-50bd-42ab-b9f4-4c1184b640cb,2025-10-01-00-00
4,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[1031535cf2a1315422fd05d321349bcd3c3ffc04, 478...",0.293285,6.0,0.160243,None,None,None,None,219d253f-bef4-4039-84b2-ed55f009cc43,2025-10-01-00-00


In [14]:
import pandas as pd

pd.set_option("display.max_rows", None)
null_columns = ddf.isnull().sum().compute()
null_columns

buyer_d1                                           0
buyer_d7                                           0
buyer_d14                                          0
buyer_d28                                          0
buy_d7                                             0
buy_d14                                            0
buy_d28                                            0
iap_revenue_d7                                     0
iap_revenue_d14                                    0
iap_revenue_d28                                    0
registration                                   57760
retention_d1_to_d7                              9564
retention_d3_to_d7                              9564
retention_d7_to_d14                             9564
retention_d1                                    9564
retention_d3                                    9564
retentiond7                                     9564
advertiser_bundle                                  0
advertiser_category                           

In [16]:
# Calcular porcentaje de nulos
null_pct = (null_columns / nrows * 100).sort_values(ascending=False)

# Estrategia según % de nulos:
# < 5%: imputación con media + ruido
# 5-30%: KNN o MICE
# > 30%: considerar crear feature binaria "is_missing" + imputar con mediana
null_pct

last_advertiser_action                        98.899801
advertiser_actions_action_count               98.899801
advertiser_actions_action_last_timestamp      98.899801
rev_by_adv                                    98.272170
last_buy_ts_bundle                            98.052294
last_buy                                      98.052294
last_buy_ts_category                          98.052294
ctr                                           96.405687
ctr_pct_rk                                    96.405687
whale_users_bundle_revenue_prank              96.317901
whale_users_bundle_num_buys_prank             96.317901
whale_users_bundle_total_revenue              96.317901
whale_users_bundle_total_num_buys             96.317901
iap_revenue_usd_category                      95.403119
num_buys_bundle                               95.403119
num_buys_category                             95.403119
iap_revenue_usd_bundle                        95.403119
iap_revenue_usd_category_bottom_taxonomy      95

In [23]:
ddf['whale_users_bundle_revenue_prank'][14].compute()
ddf['whale_users_bundle_total_revenue'][14].compute()

[('163ffe25c9eb6e1d5702e6ae5e539f9b570bbdf1', 401.55124261939136)]

In [27]:
ddf['buyer_d7'].value_counts().compute()

df_pos = ddf[ddf['buyer_d7'] == 1]
df_neg = ddf[ddf['buyer_d7'] == 0]

In [32]:
n_pos = df_pos.shape[0].compute()
n_pos

5156

In [49]:
n_neg = df_neg.shape[0].compute()

In [50]:
n_neg

116731

In [36]:
df_neg_sampled = df_neg.sample(frac=(n_pos*4) / df_neg.shape[0].compute(), random_state=42)
df_neg_sampled.shape[0].compute()

20624

In [ ]:
def undersample_partition(df, target_col="buyer_d7", majority=0, minority=1, ratio=4.0):
    pos = df[df[target_col] == minority]
    neg = df[df[target_col] == majority]
    
    n_pos = len(pos)
    n_keep_neg = int(n_pos * ratio)
    
    neg_sampled = neg.sample(n=n_keep_neg, random_state=42) if len(neg) > n_keep_neg else neg
    
    return dd.concat([pos, neg_sampled])

df_balanced = ddf.map_partitions(undersample_partition)

In [48]:
df_balanced.shape[0].compute()

25780

iap_revenue_d7 -> Variable de quants diners gastara en una setmana.  
buyer_d7 -> Variable de si l'usuari ha comprat dins de l'aplicació en una setmana (1 = sí, 0 = no)
